## ChatGPT alternatives 

In [7]:
!nvidia-smi

Fri May  5 21:10:58 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.108.03   Driver Version: 510.108.03   CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:01:00.0 Off |                  N/A |
| 30%   49C    P8    26W / 350W |   1151MiB / 24576MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA GeForce ...  Off  | 00000000:02:00.0 Off |                  N/A |
| 30%   

: 

1. StableLM
- https://github.com/Stability-AI/StableLM

In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, StoppingCriteria, StoppingCriteriaList


/home/chengyu.huang/anaconda3/envs/sbert/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
cache_dir = '/data/hf_cache'
tokenizer = AutoTokenizer.from_pretrained("stabilityai/stablelm-tuned-alpha-7b",cache_dir=cache_dir)
model = AutoModelForCausalLM.from_pretrained("stabilityai/stablelm-tuned-alpha-7b",cache_dir=cache_dir)
model.half().cuda()

Loading checkpoint shards: 100%|██████████| 4/4 [02:48<00:00, 42.07s/it]


In [7]:
class StopOnTokens(StoppingCriteria):
    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs) -> bool:
        stop_ids = set([50278, 50279, 50277, 1, 0])
        return input_ids[0][-1] in stop_ids

In [65]:
system_prompt = """<|SYSTEM|># StableLM Tuned (Alpha version)
- StableLM is a helpful and harmless open-source AI language model developed by StabilityAI.
- StableLM is excited to be able to help the user, but will refuse to do anything that could be considered harmful to the user.
- StableLM is more than just an information source, StableLM is also able to write poetry, short stories, and make jokes.
- StableLM will refuse to participate in anything that could harm a human.
"""
user_prompt = 'Please summarize what a large language model can do in ten bullet points.'

In [66]:
prompt = f"{system_prompt}<|USER|>{user_prompt}<|ASSISTANT|>"

inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
tokens = model.generate(
  **inputs,
  max_new_tokens=512,
  temperature=0.1,
  do_sample=True,
  stopping_criteria=StoppingCriteriaList([StopOnTokens()]),
  repetition_penalty=1.1
)
res = tokenizer.decode(tokens[0], skip_special_tokens=True)
print(res.replace(system_prompt.strip('<|SYSTEM|>'),'').replace(user_prompt,''))

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


* Provide natural language processing capabilities for applications such as chatbots, virtual assistants, and customer service
* Improved text generation with better grammar and style control
* Enhanced machine translation with improved accuracy and fluency
* Better understanding of context and relationships between words
* More accurate sentiment analysis and classification of text data
* Increased ability to generate coherent and contextually relevant text
* Support for knowledge management and organization of large amounts of text data
* Ability to perform tasks such as text summarization, paraphrasing, and question answering
* Better integration with other tools and systems
* Potential for increased use in legal and educational contexts


## 2. StableVicunna
- need more recent transformer library

- !pip -q install git+https://github.com/huggingface/transformers # need to install from github
- !pip install -q datasets loralib sentencepiece 
- !pip -q install bitsandbytes accelerate
- when encounter bitsandbytes errors see [link](https://github.com/TimDettmers/bitsandbytes/issues/156#issuecomment-1474056975)

In [2]:
from transformers import LlamaTokenizer, LlamaForCausalLM, GenerationConfig, pipeline
import torch

In [3]:
## load heBloke/stable-vicuna-13B-HF ## some user already converted it to HF format 
cache_dir = '/data/hf_cache'

tokenizer = LlamaTokenizer.from_pretrained("TheBloke/stable-vicuna-13B-HF",cache_dir=cache_dir)

base_model = LlamaForCausalLM.from_pretrained(
    "TheBloke/stable-vicuna-13B-HF",
    load_in_8bit=True,
    device_map='auto',    ## here, i am in a cpu only environment, so commneted out this 
    cache_dir=cache_dir,
    #torch_dtype  = torch.float16   ## here, i am in a cpu only environment, so load in 16bit  
)

Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.
/home/chengyu.huang/anaconda3/envs/sbert/lib/python3.8/site-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: /home/chengyu.huang/anaconda3/envs/sbert did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/home/chengyu.huang/anaconda3/envs/sbert/lib/python3.8/site-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('vs/workbench/api/node/extensionHostProcess')}
  warn(msg)
/home/chengyu.huang/anaconda3/envs/sbert/lib/python3.8/site-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /home/chengyu.huang/anaconda3/envs/sbert/lib/python3.8/site-packages/bitsandbytes/libbitsandbytes_cpu.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
ERROR: /home/chengyu.huang/anaconda3/envs/sbert/bin/python: undefined symbol: cudaRuntimeGetVersion
CUDA SETUP: libcudart.so path is None
CUDA SETUP: Is seems that your cuda installation is not in your path. See https://github.com/TimDettmers/bitsandbytes/issues/85 for more information.
CUDA SETUP: CUDA version lower than 11 are currently not supported for LLM.int8(). You will be only to use 8-bit optimizers and quantization routines!!
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: 

Loading checkpoint shards: 100%|██████████| 3/3 [00:35<00:00, 11.90s/it]


In [4]:
pipe = pipeline(
    "text-generation",
    model=base_model, 
    tokenizer=tokenizer, 
    max_length=512,
    temperature=0.7,
    top_p=0.95,
    repetition_penalty=1.15
)

Xformers is not installed correctly. If you want to use memorry_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


The prompt & response

In [5]:
import json
import textwrap

human_prompt = 'What is the meaning of life?'

def get_prompt(human_prompt):
    prompt_template=f"### Human: {human_prompt} \n### Assistant:"
    return prompt_template

print(get_prompt('What is the meaning of life?'))

def remove_human_text(text):
    return text.split('### Human:', 1)[0]

def parse_text(data):
    for item in data:
        text = item['generated_text']
        assistant_text_index = text.find('### Assistant:')
        if assistant_text_index != -1:
            assistant_text = text[assistant_text_index+len('### Assistant:'):].strip()
            assistant_text = remove_human_text(assistant_text)
            wrapped_text = textwrap.fill(assistant_text, width=100)
            print(wrapped_text)

data = [{'generated_text': '### Human: What is the capital of England? \n### Assistant: The capital city of England is London.'}]
parse_text(data)

### Human: What is the meaning of life? 
### Assistant:
The capital city of England is London.


#### Run some test examples 

In [7]:
#%%time 
raw_output = pipe(get_prompt('What are the difference between Llamas, Alpacas and Vicunas?'))
parse_text(raw_output)

Llamas, alpacas, and vicuñas are all members of the camelid family. However, there are some
differences between them:  1. Appearance: Llamas have a stockier build than alpacas and vicuñas,
which are more slender. Llamas also have longer fur that can be used for clothing or other products.
2. Size: Llamas are generally larger than both alpacas and vicuñas. They can weigh up to 500 pounds
(227 kg), while alpacas typically weigh around 150-200 pounds (68-91 kg) and vicuñas around 130-140
pounds (59-63 kg).  3. Temperament: Llamas tend to be calmer and less skittish than alpacas and
vicuñas. This makes them easier to handle and train for tasks like packing or pulling carts.  4.
Uses: Llamas are often used as pack animals in South America, while alpacas and vicuñas are valued
for their soft, luxurious fleece, which is used to make high-end clothing and accessories.  Overall,
each species has its own unique characteristics and uses within their respective regions.


In [8]:
raw_output = pipe(get_prompt('What drives inflation in the past year?'))
parse_text(raw_output)

There are several factors that can drive inflation, including changes in supply and demand for goods
and services, fluctuations in currency exchange rates, and government policies such as taxes or
subsidies. In recent years, rising oil prices have also been a major driver of inflation due to
their impact on transportation costs and other industries. Additionally, economic growth and job
creation can lead to increased consumer spending, which can further fuel inflation. However, it is
important to note that there are many complex factors at play when it comes to inflation, and its
causes can vary depending on the specific circumstances.


In [10]:
raw_output = pipe(get_prompt('What is current account balance in a macro-economic setting?'))
parse_text(raw_output)

In a macro-economic setting, the current account balance refers to the difference between a
country's imports and exports of goods and services. It measures the net flow of foreign currency
into or out of a country's economy. A positive current account balance indicates that a country has
more assets than liabilities abroad, while a negative current account balance suggests that a
country owes more money to foreign creditors than it receives from them.


In [13]:
raw_output = pipe(get_prompt('Please summarize what a large language model can do in ten bullet points.'))
parse_text(raw_output)

Here are ten things that a large language model can do: 1. Generate text based on prompts or inputs.
2. Answer questions and provide information about various topics. 3. Translate between languages. 4.
Summarize long pieces of text. 5. Write creative stories, poems, and other forms of literature. 6.
Create natural-sounding speech from written text. 7. Classify objects and images using computer
vision techniques. 8. Predict the next word or sentence given a context. 9. Play games like chess or
Go by analyzing board positions. 10. Perform data analysis tasks such as clustering and anomaly
detection.


In [15]:
raw_output = pipe(get_prompt('Please write an essay explaining how intrest rate affect inflation.'))
parse_text(raw_output)

The relationship between interest rates and inflation is complex, but there are a few key factors to
consider. When the Federal Reserve raises interest rates, it can have both positive and negative
effects on inflation. On one hand, higher interest rates make borrowing more expensive for consumers
and businesses, which can lead to less spending and investment in the economy. This decrease in
economic activity can slow down overall growth and potentially lead to lower inflation over time.
However, if the Fed raises interest rates too much or too quickly, it can also cause a recession,
which would increase unemployment and reduce wages, leading to higher inflation in the long run.  On
the other hand, when the Fed lowers interest rates, it can stimulate economic activity by making
borrowing cheaper and encouraging people to spend and invest more. This increased demand for goods
and services can drive up prices and lead to higher inflation. Additionally, lower interest rates
can encourage 

## 3. OpenAssistant 30b
- we are using HF version converted by Bloke [here]()

In [1]:
from transformers import LlamaTokenizer, LlamaForCausalLM, GenerationConfig, pipeline
import torch

/home/chengyu.huang/anaconda3/envs/sbert/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
cache_dir = '/data/hf_cache'

tokenizer = LlamaTokenizer.from_pretrained("TheBloke/OpenAssistant-SFT-7-Llama-30B-HF",cache_dir=cache_dir)

base_model = LlamaForCausalLM.from_pretrained(
    "TheBloke/OpenAssistant-SFT-7-Llama-30B-HF",
    load_in_8bit=True,
    device_map='auto',    ## here, i am in a cpu only environment, so commneted out this 
    cache_dir=cache_dir,
    #torch_dtype  = torch.float16   ## here, i am in a cpu only environment, so load in 16bit  
)

Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.
/home/chengyu.huang/anaconda3/envs/sbert/lib/python3.8/site-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: /home/chengyu.huang/anaconda3/envs/sbert did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/home/chengyu.huang/anaconda3/envs/sbert/lib/python3.8/site-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('vs/workbench/api/node/extensionHostProcess')}
  warn(msg)
/home/chengyu.huang/anaconda3/envs/sbert/lib/python3.8/site-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /home/chengyu.huang/anaconda3/envs/sbert/lib/python3.8/site-packages/bitsandbytes/libbitsandbytes_cpu.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
ERROR: /home/chengyu.huang/anaconda3/envs/sbert/bin/python: undefined symbol: cudaRuntimeGetVersion
CUDA SETUP: libcudart.so path is None
CUDA SETUP: Is seems that your cuda installation is not in your path. See https://github.com/TimDettmers/bitsandbytes/issues/85 for more information.
CUDA SETUP: CUDA version lower than 11 are currently not supported for LLM.int8(). You will be only to use 8-bit optimizers and quantization routines!!
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: 

Loading checkpoint shards: 100%|██████████| 7/7 [06:15<00:00, 53.65s/it]


In [3]:
pipe = pipeline(
    "text-generation",
    model=base_model, 
    tokenizer=tokenizer, 
    max_length=512,
    temperature=0.7,
    top_p=0.95,
    repetition_penalty=1.15
)

Xformers is not installed correctly. If you want to use memorry_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


In [14]:
raw_output = pipe('Please write an essay explaining how intrest rate affect inflation.<|assistant|>:')
print(raw_output)

In [10]:
raw_output = pipe('<|prompter|>Please write an essay explaining how intrest rate affect inflation.<|assistant|>:')
print(raw_output[0]['generated_text'])


[{'generated_text': '<|prompter|>Please write an essay explaining how intrest rate affect inflation.<|assistant|>:\n\nInflation is the increase in prices of goods and services over a period of time, while interest rates are the cost of borrowing money from banks or other financial institutions. The relationship between these two economic indicators is complex and multifaceted, but there are several key ways that interest rates can impact inflation:\n\n1. Monetary policy: Central banks use monetary policy to control inflation by adjusting interest rates. When inflation rises above a certain level, central banks may raise interest rates to reduce demand for credit and slow down economic growth. This can help bring inflation back under control. Conversely, when inflation falls below target levels, central banks may lower interest rates to stimulate economic activity and boost inflation.\n2. Consumer spending: Interest rates also influence consumer behavior and spending patterns. Higher in